In [ ]:
# Import all necessary libraries
from traffic.data.samples import belevingsvlucht
import traffic
from traffic.data import opensky
import datetime
import trino
from pyopensky.trino import Trino
from pyopensky.rest import REST
import pandas as pd
import os

In [ ]:
# Define the start and stop time of the data request
start_time_str       = '2022-11-01T00:00:00Z'
stop_time_str        = '2022-11-01T01:00:00Z'
plane_callsign       = "EZY158T"
query_limit          = 1e1

# Convert the start and stop time to POSIX time
start_time_posix    = int(datetime.datetime.fromisoformat(start_time_str).timestamp())
end_time_posix      = int(datetime.datetime.fromisoformat(stop_time_str).timestamp())

# Define the SQL query to get the data from the OpenSky database
sql_query  =    """SELECT icao24, estdepartureairport, estarrivalairport, callsign, track FROM flights_data4 
                    WHERE day = """ + str(start_time_posix) + """
                    LIMIT """ + str(round(query_limit)) 

print(sql_query)


In [ ]:
# Pass the query to the database and get the result
db = Trino()
print(sql_query)
result_df = db.query(sql_query, cached=False)

# Save the result to a pickle file
timestamp = datetime.datetime.fromisoformat(start_time_str.replace("Z", "")).strftime("%Y%m%d")
output_dir = os.path.expanduser("~/Data")
os.makedirs(output_dir, exist_ok=True)
filename = os.path.join(output_dir, f"result_df_{timestamp}.pkl")
result_df.to_pickle(filename)
print("Saved result_df to", filename)

In [1]:
import folium

# Find the first flight with a non-empty track to center the map
valid_tracks = result_df[result_df['track'].apply(lambda x: x is not None and len(x) > 0)]
if valid_tracks.empty:
    raise ValueError("No valid flight tracks found in result_df.")

# Use the first point of the first valid flight to center the map.
first_point = valid_tracks.iloc[0]['track'][0]
map_center = [first_point.latitude, first_point.longitude]  # using the correct attribute names

# Create the map
m = folium.Map(location=map_center, zoom_start=6)

# Iterate over each flight and add its flight path as a PolyLine
for _, row in result_df.iterrows():
    track = row['track']
    if not track:
        continue
    # Convert each point in the track to [latitude, longitude]
    line_coords = [[point.latitude, point.longitude] for point in track]
    folium.PolyLine(line_coords, color="blue", weight=2.5, opacity=1).add_to(m)

# Display the map
m

NameError: name 'result_df' is not defined